In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob

In [2]:
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [3]:
def func(p):
    path=glob.glob(p)
    data=[]
    for file in path:
        try:
            img=cv2.imread(file)
            img=cv2.resize(img,(600,500))
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray,scaleFactor=1.25,minNeighbors=5,minSize=(30, 30))
            for x,y,w,h in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(100,200,0),3)
                face=img[y:y+h,x:x+w,:]
                face=cv2.resize(face,(100,100))
                data.append(face)
            #cv2.imshow('sdf',img)
            #if cv2.waitKey(300) & 0xFF == 27:break
        except: break
    #cv2.destroyAllWindows()
    return data


In [4]:
np.save('a.npy',func("C:/Users/pytho/Desktop/Train/Non Mask/*.jpg"))
no_mask=np.load('a.npy')
np.save('b.npy',func("C:/Users/pytho/Desktop/Train/Mask/*.jpg"))
mask=np.load('b.npy')

In [5]:
print(mask.shape, no_mask.shape)
print(len(no_mask))

(510, 100, 100, 3) (1101, 100, 100, 3)
1101


In [6]:
mask=mask.reshape(len(mask),100*100*3)
no_mask=no_mask.reshape(len(no_mask),100*100*3)

In [7]:
no_mask.shape

(1101, 30000)

In [8]:
x=np.r_[no_mask,mask]
x.shape[0]

1611

In [9]:
y=np.ones(x.shape[0])
y[:len(no_mask)]=0.0

In [10]:
#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30)
x_train=x
y_train=y
x_train.shape

(1611, 30000)

In [11]:
np.save('a.npy',func("C:/Users/pytho/Desktop/Test/Non Mask/*.jpg"))
no_mask=np.load('a.npy')
np.save('b.npy',func("C:/Users/pytho/Desktop/Test/Mask/*.jpg"))
mask=np.load('b.npy')

In [12]:
print(mask.shape, no_mask.shape)
print(len(mask))

(13, 100, 100, 3) (98, 100, 100, 3)
13


In [13]:
mask=mask.reshape(len(mask),100*100*3)
no_mask=no_mask.reshape(len(no_mask),100*100*3)

In [14]:
m=np.r_[no_mask,mask]
m.shape[0]

111

In [15]:
n=np.ones(m.shape[0])
n[:len(no_mask)]=0.0

In [16]:
x_test=m
y_test=n

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
r = RandomForestClassifier(n_estimators=600)
r.fit(x_train,y_train)
y_pred=r.predict(x_test)
accuracy_score(y_test,y_pred)

In [19]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(x_train,y_train)
predictions = logmodel.predict(x_test)
accuracy_score(y_test,y_pred)

C:\Users\pytho\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9819819819819819

In [20]:
m=SVC()
m.fit(x_train,y_train)
y_pred=m.predict(x_test)
accuracy_score(y_test,y_pred)

0.990990990990991

In [21]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99        98
         1.0       1.00      0.92      0.96        13

    accuracy                           0.99       111
   macro avg       0.99      0.96      0.98       111
weighted avg       0.99      0.99      0.99       111



In [28]:
lol=1
font=cv2.FONT_HERSHEY_COMPLEX
d={1:'MASK',0:'No Mask'}

In [30]:
cap=cv2.VideoCapture(r'Videos/AA.mp4')
#cap=cv2.VideoCapture(r'Downloads/fucking_awesome.mp4')
#cap=cv2.VideoCapture("C:/Users/pytho/Downloads/Breda verdeeld over dragen mondkapjes Omroep Brabant_720P HD.mp4")
#cap=cv2.VideoCapture(r'Videos/video.mp4')
#cap=cv2.VideoCapture(0)
while (cap.isOpened()):
    flag,img=cap.read()
    try:
        img=cv2.resize(img,(800,500))
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        #faces=face_cascade.detectMultiScale(gray,1.1,4)
        faces = face_cascade.detectMultiScale(gray,scaleFactor=1.18,minNeighbors=5,minSize=(30, 30))
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),4)
            face=img[y:y+h,x:x+w,:]
            face=cv2.resize(face,(100,100))
            face=face.reshape(1,-1)
            pred=logmodel.predict(face)[0]
            n=d[int(pred)]
            cv2.putText(img,n,(x,y),font,1,(0,0,255),2)
        cv2.imshow('hell0_world',img)
        if cv2.waitKey(lol) & 0xFF==27:
            break
    except: break
cap.release()
cv2.destroyAllWindows()